In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
import psycopg2
import random

In [2]:
def con_db_caio():
    conn = psycopg2.connect(host="35.247.202.234",database="dw",user="consulta",password="teste123")
    return conn

In [5]:
def grafico_risco_escola_dimensoes_barras2():
    query = """select * from public.dimensoes_est de
    inner join escolas.aluno a 
    on de.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
    df = pd.read_sql(query, con_db_caio())

    df_temp = df.groupby('E_ESCC')['id'].count()
    df_temp = df_temp.reset_index()
    df_temp.columns = ['Classificação', 'Quant. Estudantes']
    df_temp['Porcentagem'] = (df_temp['Quant. Estudantes'] / df_temp['Quant. Estudantes'].sum()) * 100
    df_temp
    
    df_temp['class'] = 'R' + df_temp['Classificação'].astype(str)
    i = 1
    for index, row in df_temp.iterrows():
        # print(row['class'])
        if (i == 1):
            c1 = row['class']
            i = i + 1
        else:
            c2 = row['class']

    df_dim_graph = pd.DataFrame(columns=['Dimensão', 'R1', 'R2', 'R3'])

    df_for = ['E_ESCC', 'E_PROFC', 'E_FAMC', 'E_COMC', 'E_ESTC']

    for x in df_for:
        # print(x)
        df_temp = df.groupby(x)['id'].count()
        df_temp = df_temp.reset_index()
        df_temp.columns = ['Classificação', 'Quant. Estudantes']
        df_temp['Dimensao'] = x
        df_temp['Porcentagem'] = (df_temp['Quant. Estudantes'] / df_temp['Quant. Estudantes'].sum()) * 100
        # df_dim2.loc[index,nomeColuna] = df_reun['Result'].iloc[0]
        df_temp['class'] = 'R' + df_temp['Classificação'].astype(str)
        if (len(df_temp) == 3):
            c1 = 'R1'
            c2 = 'R2'
            c3 = 'R3'
            df_dim_graph = df_dim_graph.append({'Dimensão': x,
                                                c1: df_temp['Porcentagem'].iloc[0],
                                                c2: df_temp['Porcentagem'].iloc[1],
                                                c3: df_temp['Porcentagem'].iloc[2]}, ignore_index=True)
        elif (len(df_temp) == 2):

            i = 1
            for index, row in df_temp.iterrows():
                # print(row['class'])
                if (i == 1):
                    c1 = row['class']
                    i = i + 1
                else:
                    c2 = row['class']
            df_dim_graph = df_dim_graph.append({'Dimensão': x,
                                                c1: df_temp['Porcentagem'].iloc[0],
                                                c2: df_temp['Porcentagem'].iloc[1],
                                                }, ignore_index=True)

        else:
            for index, row in df_temp.iterrows():
                c1 = row['class']
            df_dim_graph = df_dim_graph.append({'Dimensão': x,
                                                c1: df_temp['Porcentagem'].iloc[0],
                                                }, ignore_index=True)
    df_dim_graph
    df_dim_graph = df_dim_graph.fillna(0)
    df_dim_graph = df_dim_graph.sort_values('R3', ascending=True)
    df_dim_graph.round(2)

    def trunc(values, decs=0):
        return np.trunc(values * 10 ** decs) / (10 ** decs)

    x_data = df_dim_graph[["R3", "R2", "R1"]].to_numpy()
    x_data = trunc(x_data, decs=2)
    x_data[np.isnan(x_data)] = 0

    y_data = list(df_dim_graph['Dimensão'])

    # --
    top_labels = ['Alto', 'Médio', 'Baixo']
        
    colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
              'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
              'rgba(190, 192, 213, 1)']

    # x_data = [[8.06, 12.90, 79.03,],
    #          [8.06, 27.41, 64.51,],
    #          [9.67, 24.19, 66.12,],
    #          [16.12, 11.29, 72.58,],
    #         [33.87, 33.87, 32.23,]]

    # y_data = ['Estudante<br>Estudante',
    #          'Escudante<br>Profissionais da' +
    #          '<br>Escola', 'Estudante' +
    #          '<br>Familia',
    #          'Estudante<br>Comunidade', 'Estudante<br>Escola']

    fig = go.Figure()

    for i in range(0, len(x_data[0])):
        for xd, yd in zip(x_data, y_data):
            fig.add_trace(go.Bar(
                x=[xd[i]], y=[yd],
                orientation='h',
                marker=dict(
                    color=colors[i],
                    line=dict(color='rgb(248, 248, 249)', width=1)
                )
            ))

    fig.update_layout(
        xaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=False,
            showline=False,
            showticklabels=False,
            zeroline=False,
        ),
        barmode='stack',
        paper_bgcolor='rgb(248, 248, 255)',
        plot_bgcolor='rgb(248, 248, 255)',
        margin=dict(l=120, r=10, t=140, b=80),
        showlegend=False,
    )

    annotations = []

    for yd, xd in zip(y_data, x_data):
        # labeling the y-axis

        annotations.append(dict(xref='paper', yref='y',
                                x=0.14, y=yd,
                                xanchor='right',
                                text=str(yd),
                                font=dict(family='Arial', size=14,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False, align='right'))
        # labeling the first percentage of each bar (x_axis)
        if (xd[0] != 0):
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] / 2, y=yd,
                                    #text=str(xd[0]) + '%',
                                    text='',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
        else:
            annotations.append(dict(xref='x', yref='y',
                                    x=xd[0] / 2, y=yd,
                                    text='',
                                    font=dict(family='Arial', size=14,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))

        # labeling the first Likert scale (on the top)
        if yd == y_data[-1]:
            annotations.append(dict(xref='x', yref='paper',
                                    x=xd[0] / 2, y=1.1,
                                    text=top_labels[0],
                                    font=dict(family='Arial', size=14,
                                              color='rgb(67, 67, 67)'),
                                    showarrow=False))
        space = xd[0]
        for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            if (xd[i] != 0):
                annotations.append(dict(xref='x', yref='y',
                                        x=space + (xd[i] / 2), y=yd,
                                        #text=str(xd[i]) + '%',
                                        text='',
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(248, 248, 255)'),
                                        showarrow=False))
            else:
                annotations.append(dict(xref='x', yref='y',
                                        x=space + (xd[i] / 2), y=yd,
                                        text='',
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(248, 248, 255)'),
                                        showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i] / 2), y=1.1,
                                        text=top_labels[i],
                                        font=dict(family='Arial', size=14,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

    fig.update_layout(annotations=annotations,title={
        'text': "Risco por Dimensão em %",
        'y':0.9,
        'x':0.5,
        #'xanchor': 'center',
        'yanchor': 'top'})

    return fig

In [6]:
v = grafico_risco_escola_dimensoes_barras2()
v.show()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

C:\Users\emanuelqueiroga\AppData\Local\Temp\ipykernel_1820\322231579.py:44: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\emanuelqueiroga\AppData\Local\Temp\ipykernel_1820\322231579.py:44: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\emanuelqueiroga\AppData\Local\Temp\ipykernel_1820\322231579.py:44: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\emanuelqueiroga\AppData\Local\Temp\ipykernel_1820\322231579.py:44: FutureWarning:

The frame.app

In [11]:
 query = """select * from public.fatores_est fe
    inner join escolas.aluno a 
    on fe.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
df = pd.read_sql(query, con_db_caio())
#df.replace(0, 1, inplace=True)
df = df.rename({'E_ESC1C': 'X_Cond. Mat. Escola', 'E_ESC2C': 'X_Cond. Mat. Aluno'}, axis=1)
df = df.rename({'E_PROF1C': 'X_Inflexibilidade Pedagógica', 'E_PROF2C': 'X_Qual. Pedagógica'}, axis=1)
df = df.rename({'E_FAM1C': 'X_Suporte Familiar', 'E_FAM2C': 'X_Grav./Par. e Atividades Domésticas de Cuidado'}, axis=1)
df = df.rename({'E_COM1C': 'X_Med. Soc. e Cont. de Violência', 'E_COM2C': 'X_Dist. Esc.-Comunidade', 'E_COM3C': 'X_Acessibilidade/Freq. Esc.'}, axis=1)
df = df.rename({'E_EST1C': 'X_Significados da Esc/Eng. do Est.', 'E_EST2C': 'X_Asp. Emoc. e Afetivos', 'E_EST3C': 'X_Repr. e Distorção Idade-Série.'}, axis=1)
df_dim = pd.DataFrame()
df_dim = pd.DataFrame()

df_col = df.loc[:, df.columns.str.startswith("X_")]
#df_col = df_col.loc[:, df_col.columns.str.endswith("C")]
df_for = list(df_col)
list(df_col)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



['X_Cond. Mat. Escola',
 'X_Cond. Mat. Aluno',
 'X_Inflexibilidade Pedagógica',
 'X_Qual. Pedagógica',
 'X_Suporte Familiar',
 'X_Grav./Par. e Atividades Domésticas de Cuidado',
 'X_Med. Soc. e Cont. de Violência',
 'X_Dist. Esc.-Comunidade',
 'X_Acessibilidade/Freq. Esc.',
 'X_Significados da Esc/Eng. do Est.',
 'X_Asp. Emoc. e Afetivos',
 'X_Repr. e Distorção Idade-Série.']

In [18]:
def grafico_risco_escola_fatores_barras():
    query = """select * from public.fatores_est fe
    inner join escolas.aluno a 
    on fe.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
    df = pd.read_sql(query, con_db_caio())
    #df.replace(0, 1, inplace=True)
    df = df.rename({'E_ESC1C': 'X_Cond. Mat. Escola', 'E_ESC2C': 'X_Cond. Mat. Aluno'}, axis=1)
    df = df.rename({'E_PROF1C': 'X_Inflexibilidade Pedagógica', 'E_PROF2C': 'X_Qual. Pedagógica'}, axis=1)
    df = df.rename({'E_FAM1C': 'X_Suporte Familiar', 'E_FAM2C': 'X_Grav./Par. e Atividades Domésticas de Cuidado'}, axis=1)
    df = df.rename({'E_COM1C': 'X_Med. Soc. e Cont. de Violência', 'E_COM2C': 'X_Dist. Esc.-Comunidade', 'E_COM3C': 'X_Acessibilidade/Freq. Esc.'}, axis=1)
    df = df.rename({'E_EST1C': 'X_Significados da Esc/Eng. do Est.', 'E_EST2C': 'X_Asp. Emoc. e Afetivos', 'E_EST3C': 'X_Repr. e Distorção Idade-Série.'}, axis=1)
    
    df_dim = pd.DataFrame()
    df_dim = pd.DataFrame()
    
    df_col = df.loc[:, df.columns.str.startswith("X_")]
    #df_col = df.loc[:, df.columns.str.startswith("E_")]
    #df_col = df_col.loc[:, df_col.columns.str.endswith("C")]
    df_for = list(df_col)
    # df_for = ['E_ESCC', 'E_PROFC', 'E_FAMC', 'E_COMC', 'E_ESTC']

    for x in df_for:
        # print(x)
        df_temp = df.groupby(x)['id'].count()
        df_temp = df_temp.reset_index()
        df_temp.columns = ['Classificação', 'Quant. Estudantes']
        df_temp['Dimensão'] = x
        df_temp['Porcentagem'] = (df_temp['Quant. Estudantes'] / df_temp['Quant. Estudantes'].sum()) * 100
        frames = [df_dim, df_temp]
        df_dim = pd.concat(frames)

    df_dim = df_dim.sort_values(by=['Classificação', 'Porcentagem'], ascending=[False, False])

    df_dim['Risco'] = df_dim['Classificação']
    df_dim['Risco'] = df_dim['Risco'].astype(str)
    df_dim['Risco'] = df_dim['Risco'].replace('0', 'Risco Baixo')
    df_dim['Risco'] = df_dim['Risco'].replace('1', 'Risco Baixo')
    df_dim['Risco'] = df_dim['Risco'].replace('2', 'Risco Médio')
    df_dim['Risco'] = df_dim['Risco'].replace('3', 'Risco Alto')
    df_dim['Dimensão'] = df_dim['Dimensão'].str.replace(r'X_', '')
    df_dim = df_dim.rename({'Dimensão': 'Fatores'}, axis=1)
    df_dim = df_dim[df_dim.Risco != '0']
    df_dim
    fig = px.bar(df_dim, x="Fatores", y="Quant. Estudantes", color='Risco')
    return fig
    #return df_dim

In [19]:
f = grafico_risco_escola_fatores_barras()
f.show()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



In [3]:
query = """select * from public.fatores_est fe
    inner join escolas.aluno a 
    on fe.id_aluno = a.id_aluno 
    inner join escolas.turma t 
    on a.id_turma = t.id_turma 
    inner join escolas.escola e 
    on t.id_escola = e.cod_escola"""
df = pd.read_sql(query, con_db_caio())
df.replace(0, np.nan, inplace=True)
df = df.dropna()
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,id_aluno,E_ESC1V,E_ESC2V,E_PROF1V,E_PROF2V,E_FAM1V,E_FAM2V,E_COM1V,E_COM2V,...,conveniada_poder_publico,regulamentacao_conselho,porte,etapas_e_modalidade_ensino,outras_ofertas,cod_regiao,micro,meso,regiao,cod_estado
